In [14]:
import tensorflow as tf
import tf2onnx
import onnx
import warnings
import numpy as np
import os

# Suppress all warnings
warnings.filterwarnings('ignore')

In [7]:
keras_model= tf.keras.models.load_model("../second_hand_car_price_regression_model.keras")
keras_model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ normalization_1 (Normalization) │ (None, 10)             │            21 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 256)            │         2,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 137,753 (538.11 KB)

 Trainable params: 68,865 (269.00 KB)

 Non-trainable params: 21 (88.00 B)

 Optimizer params: 68,867 (269.02 KB)

In [37]:
input= np.array([6.83685e+05, 7.54601e+05, 6.00000e+00, 7.05110e+04, 2.00000e+00,
        9.00000e+00, 1.50000e+01, 1.85000e+02, 1.17000e+02, 7.90000e+01])
input.shape, input.dtype

((10,), dtype('float64'))

In [13]:
keras_model.predict(tf.reshape(input, (1,-1))) # model is able to predict

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step


array([[475553.6]], dtype=float32)

In [25]:
def get_filesize(filepath, model_type):
    file_sizekb = os.path.getsize(filepath)/1024
    file_sizemb = file_sizekb / 1024
    print(f"{model_type} Model size in Kb : {file_sizekb}")
    print(f"{model_type} Model size in Mb : {file_sizemb}")

## keras model file size
get_filesize("../second_hand_car_price_regression_model.keras", "Keras")

Keras Model size in Kb : 835.2333984375
Keras Model size in Mb : 0.8156576156616211


In [34]:
keras_model.output_names=['output']

## Convert keras model to Onnx

In [39]:
os.getcwd()

'd:\\tensorflow\\second_hand_car_price_prediction\\onnx_model'

In [58]:
input_signature = [tf.TensorSpec([None, 10], tf.float32)] #define the input schema


onnx_model, _ = tf2onnx.convert.from_keras(keras_model, input_signature, opset=13)


onnx.save(onnx_model, "second_hand_car_price_model.onnx") # save the model

rewriter <function rewrite_constant_fold at 0x00000214EFB00360>: exception `np.cast` was removed in the NumPy 2.0 release. Use `np.asarray(arr, dtype=dtype)` instead.


## Looad onnx model and inferencing

In [ ]:
# second_hand_car_price_model.onnx

In [59]:
import onnxruntime as ort

session = ort.InferenceSession("second_hand_car_price_model.onnx", providers=["CPUExecutionProvider"]) #create session 

input_name = session.get_inputs()[0].name

In [63]:
input_array= np.array([6.83685e+05, 7.54601e+05, 6.00000e+00, 7.05110e+04, 2.00000e+00,
       9.00000e+00, 1.50000e+01, 1.85000e+02, 1.17000e+02, 7.90000e+01]).reshape(1,-1)

input_array.shape, input_array #model input= [1,10]

((1, 10),
 array([[6.83685e+05, 7.54601e+05, 6.00000e+00, 7.05110e+04, 2.00000e+00,
         9.00000e+00, 1.50000e+01, 1.85000e+02, 1.17000e+02, 7.90000e+01]]))

In [ ]:
output = session.run(None, {input_name: input_array.astype(np.float32)}) #inferencing

print(output)

[array([[475553.6]], dtype=float32)]


## get file size of Models

In [68]:
## keras model file size
get_filesize("../second_hand_car_price_regression_model.keras", "Keras")
print()
## onnx model file size
get_filesize("./second_hand_car_price_model.onnx", "onnx")

Keras Model size in Kb : 835.2333984375
Keras Model size in Mb : 0.8156576156616211

onnx Model size in Kb : 271.212890625
onnx Model size in Mb : 0.26485633850097656


## runtime comparison (usually onnx model run fast)

In [69]:
%time
keras_model.predict(tf.reshape(input, (1,-1))) # model is able to predict

CPU times: total: 0 ns
Wall time: 0 ns
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step


array([[475553.6]], dtype=float32)

In [70]:
%time

output = session.run(None, {input_name: input_array.astype(np.float32)}) #inferencing

print(output)


CPU times: total: 0 ns
Wall time: 0 ns
[array([[475553.6]], dtype=float32)]
